In [7]:
# Essential Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# OpenCV for Video Processing
import cv2

# MediaPipe for Pose Estimation
import mediapipe as mp

# TensorFlow and Keras
import tensorflow as tf

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Utility functions
import json

In [8]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Define paths to your dataset
video_directory = "/Users/cezar/Desktop/Team Project/AI/shotput/stage1/videos"  # Update with the actual path to your videos
output_keypoints_path = "/Users/cezar/Desktop/Team Project/AI/shotput/stage1/keypoint_data"  # Directory to save extracted keypoints
os.makedirs(output_keypoints_path, exist_ok=True)  # Create directory if it doesn't exist

# List all video files
video_files = [os.path.join(video_directory, file) for file in os.listdir(video_directory) if file.endswith('.mp4')]

print(f"Found {len(video_files)} videos.")

Found 17 videos.


I0000 00:00:1737048714.376327 14127678 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737048714.474723 14265435 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737048714.488256 14265435 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [9]:
def calculate_angle(a, b, c):
    """
    Calculate the angle between three points.
    a, b, c are (x, y) coordinates.
    """
    a = np.array(a)  # Point A
    b = np.array(b)  # Point B (vertex)
    c = np.array(c)  # Point C

    # Calculate vectors
    ab = a - b
    cb = c - b

    # Calculate the angle
    radians = np.arccos(np.dot(ab, cb) / (np.linalg.norm(ab) * np.linalg.norm(cb)))
    angle = np.degrees(radians)
    return angle

def extract_keypoints(video_path, pose_instance):
    """
    Extract keypoints and calculate relevant features from a video.
    Returns a list of features for each frame.
    """
    keypoints = []
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_instance.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant keypoints (e.g., LEFT_HIP, LEFT_KNEE, LEFT_ANKLE)
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y]

            # Calculate angles or other features
            angle = calculate_angle(left_hip, left_knee, left_ankle)
            keypoints.append(angle)

    cap.release()
    return keypoints

In [10]:
# Process and save keypoints for all videos
def process_videos(video_files, output_keypoints_path, pose_instance):
    """
    Process all videos to extract and save keypoints.
    """
    for video_path in video_files:
        # Extract keypoints
        keypoints = extract_keypoints(video_path, pose_instance)

        # Generate output file name
        video_name = os.path.basename(video_path).replace('.mp4', '.json')
        output_file = os.path.join(output_keypoints_path, video_name)

        # Save keypoints to JSON
        with open(output_file, 'w') as f:
            json.dump(keypoints, f)

        print(f"Keypoints extracted and saved for {video_name}.")

# Call the function to process all videos
process_videos(video_files, output_keypoints_path, pose)

W0000 00:00:1737048721.671147 14265434 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Keypoints extracted and saved for 1_user2.json.
Keypoints extracted and saved for 1_user1.json.
Keypoints extracted and saved for 1_user4.json.
Keypoints extracted and saved for 1_user5.json.
Keypoints extracted and saved for 1_user10.json.
Keypoints extracted and saved for 1_user7.json.
Keypoints extracted and saved for 1_user6.json.
Keypoints extracted and saved for 1_user13.json.
Keypoints extracted and saved for 0_user3.json.
Keypoints extracted and saved for 0_user12.json.
Keypoints extracted and saved for 1_user22.json.
Keypoints extracted and saved for 1_user23.json.
Keypoints extracted and saved for 1_user21.json.
Keypoints extracted and saved for 1_user8.json.
Keypoints extracted and saved for 1_user9.json.
Keypoints extracted and saved for 1_user20.json.
Keypoints extracted and saved for 1_user19.json.


In [11]:
from scipy.interpolate import interp1d

def interpolate_keypoints(keypoints, target_length=None):
    """
    Interpolates a sequence of keypoints to ensure smooth transitions.
    - keypoints: List of extracted keypoints (angles or coordinates).
    - target_length: Desired length of the output sequence (optional).
    Returns: Interpolated sequence of keypoints.
    """
    if len(keypoints) < 2:
        print("Not enough keypoints for interpolation.")
        return keypoints

    # Original indices
    original_indices = np.arange(len(keypoints))
    
    # Target indices
    if target_length:
        target_indices = np.linspace(0, len(keypoints) - 1, target_length)
    else:
        target_indices = np.arange(len(keypoints))

    # Interpolation
    interpolator = interp1d(original_indices, keypoints, kind='linear', fill_value="extrapolate")
    interpolated_keypoints = interpolator(target_indices)

    return interpolated_keypoints.tolist()

# Test interpolation on a sample keypoints file
sample_keypoints_file = os.path.join(output_keypoints_path, os.listdir(output_keypoints_path)[0])
with open(sample_keypoints_file, 'r') as f:
    keypoints = json.load(f)

# Interpolate keypoints to a fixed length (e.g., 100 frames)
interpolated_keypoints = interpolate_keypoints(keypoints, target_length=100)
print(f"Original Length: {len(keypoints)}, Interpolated Length: {len(interpolated_keypoints)}")

Original Length: 32, Interpolated Length: 100


Interpolated keypoints saved for 1_user2.json.
Interpolated keypoints saved for 1_user13.json.
Interpolated keypoints saved for 1_user8.json.
Interpolated keypoints saved for 1_user22.json.
Interpolated keypoints saved for 1_user4.json.
Interpolated keypoints saved for 1_user5.json.
Interpolated keypoints saved for 1_user19.json.
Interpolated keypoints saved for 1_user23.json.
Interpolated keypoints saved for 1_user9.json.
Interpolated keypoints saved for 1_user20.json.
Interpolated keypoints saved for 0_user3.json.
Interpolated keypoints saved for 1_user6.json.
Interpolated keypoints saved for 1_user7.json.
Interpolated keypoints saved for 1_user21.json.
Interpolated keypoints saved for 0_user12.json.
Interpolated keypoints saved for 1_user10.json.
Interpolated keypoints saved for 1_user1.json.


In [12]:
def load_keypoints_and_labels(interpolated_dir):
    """
    Load interpolated keypoints and extract labels directly from filenames.
    - interpolated_dir: Directory containing interpolated keypoints JSON files.
    Returns: X (features), y (labels).
    """
    X = []
    y = []

    for file_name in os.listdir(interpolated_dir):
        if file_name.endswith('.json'):
            file_path = os.path.join(interpolated_dir, file_name)
            
            # Load keypoints
            with open(file_path, 'r') as f:
                keypoints = json.load(f)
                X.append(keypoints)
            
            # Extract label from filename
            try:
                label = float(file_name.split('_')[0])  # Extract label from the filename (e.g., "1_user1.json")
                y.append(label)
            except ValueError:
                print(f"Skipping file {file_name}: Could not extract label.")
                continue

    X = np.array(X)
    y = np.array(y)

    return X, y

# Directory of interpolated keypoints
interpolated_dir = os.path.join(output_keypoints_path, "interpolated")

# Load features and labels
X, y = load_keypoints_and_labels(interpolated_dir)

print(f"Feature matrix shape: {X.shape}")
print(f"Labels shape: {y.shape}")

Feature matrix shape: (17, 100)
Labels shape: (17,)


In [13]:
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, TimeDistributed, LSTM, Bidirectional

In [16]:
def build_transformer_model_debug(input_shape, num_heads=4, key_dim=32, ff_dim=128, dropout_rate=0.1):
    """
    Transformer model with improved configuration for debugging.
    """
    inputs = Input(shape=input_shape)

    # Embedding Layer (optional for increasing feature dimensions)
    embedded = TimeDistributed(Dense(key_dim, activation='relu'))(inputs)

    # Multi-Head Attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(embedded, embedded)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output + embedded)

    # Feed-Forward Network
    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Dense(input_shape[-1], activation='linear')(ff_output)
    ff_output = LayerNormalization(epsilon=1e-6)(ff_output + attention_output)

    # Output Layer
    flat = Flatten()(ff_output)
    outputs = Dense(1, activation='linear')(flat)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    return model

# Rebuild and summarize the model
transformer_model_debug = build_transformer_model_debug(X_train.shape[1:])
transformer_model_debug.summary()


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 100, 32)   │         64 │ input_layer[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 32)   │     16,800 │ time_distributed… │
│ (MultiHeadAttentio… │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 100, 32)   │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 100, 32)   │          0 │ dropout_1[0][0],  │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 100, 32)   │         64 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 100, 128)  │      4,224 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 100, 128)  │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100, 1)    │        129 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 100, 32)   │          0 │ dense_2[0][0],    │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 32)   │         64 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3200)      │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │      3,201 │ flatten[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,546 (95.88 KB)

 Trainable params: 24,546 (95.88 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Validation set: X_val={X_val.shape}, y_val={y_val.shape}")

# Ensure input shape has three dimensions (samples, timesteps, features)
if len(X_train.shape) == 2:
    X_train = X_train[..., np.newaxis]  # Add a new axis for features
    X_val = X_val[..., np.newaxis]

print(f"Updated Training set: X_train={X_train.shape}")
print(f"Updated Validation set: X_val={X_val.shape}")

Training set: X_train=(13, 100), y_train=(13,)
Validation set: X_val=(4, 100), y_val=(4,)
Updated Training set: X_train=(13, 100, 1)
Updated Validation set: X_val=(4, 100, 1)


In [17]:
batch_size = 8
epochs = 300

# Train the transformer model again
history = transformer_model_debug.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 366ms/step - loss: 2.6740 - mae: 1.3517 - val_loss: 0.6728 - val_mae: 0.8192
Epoch 2/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.9585 - mae: 0.8879 - val_loss: 4.1943 - val_mae: 2.0477
Epoch 3/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 3.9611 - mae: 1.9374 - val_loss: 0.8018 - val_mae: 0.8946
Epoch 4/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.5855 - mae: 0.6713 - val_loss: 0.8583 - val_mae: 0.9254
Epoch 5/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.3572 - mae: 1.0918 - val_loss: 1.2909 - val_mae: 1.1353
Epoch 6/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.2570 - mae: 1.0606 - val_loss: 0.0022 - val_mae: 0.0375
Epoch 7/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2431 - mae: 0.3629 - val_loss: 1.0044 - val_mae: 1.0013
Epoch 8/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.1118 - mae: 0.9616 - val_loss: 0.9657 - val_mae: 0.9817
Epoch 9/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.6060 - mae:

In [18]:
# Evaluate the model on the validation set
val_loss, val_mae = transformer_model_debug.evaluate(X_val, y_val, verbose=0)

print(f"Validation Loss (MSE): {val_loss:.4f}")
print(f"Validation MAE: {val_mae:.4f}")


Validation Loss (MSE): 0.0088
Validation MAE: 0.0778


In [29]:
def test_single_video(video_path, pose_instance, model, target_length=100):
    """
    Test a single video with the trained model.
    - video_path: Path to the unseen video.
    - pose_instance: Initialized MediaPipe Pose instance.
    - model: Trained transformer model.
    - target_length: Length to interpolate keypoints.
    Returns: Predicted score.
    """
    import os

    # Step 1: Extract keypoints
    keypoints = extract_keypoints(video_path, pose_instance)
    if not keypoints:
        raise ValueError(f"No keypoints detected in video: {video_path}")

    # Step 2: Interpolate keypoints to the target length
    interpolated_keypoints = interpolate_keypoints(keypoints, target_length=target_length)

    # Step 3: Format the keypoints for model input
    X_test = np.array([interpolated_keypoints], dtype='float32')
    if len(X_test.shape) == 2:
        X_test = X_test[..., np.newaxis]  # Add feature dimension

    # Step 4: Predict the score
    predicted_score = model.predict(X_test)[0][0]

    # Step 5: Classify the predicted score into 0, 0.5, or 1
    def classify_score(prediction):
        if prediction >= 0.85:
            return 1.0
        elif prediction >= 0.70:
            return 0.5
        else:
            return 0.0

    classified_score = classify_score(predicted_score)

    return predicted_score, classified_score


# Example usage
video_path = "/Users/cezar/Desktop/Team Project/AI/shotput/stage1/videos/0_user12.mp4"  # Replace with the path to your unseen video
predicted_score, classified_score = test_single_video(video_path, pose, transformer_model_debug)
print(f"Predicted Score (Raw): {predicted_score:.2f}")
print(f"Classified Score: {classified_score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Predicted Score (Raw): 1.02
Classified Score: 1.0


In [19]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")


TensorFlow version: 2.17.0


In [4]:
import cv2
import mediapipe as mp
import os
import json
import numpy as np
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.saving import register_keras_serializable
import tensorflow.keras.backend as K

In [20]:
# from tensorflow.keras.models import load_model

# @register_keras_serializable()
# def weighted_mse(y_true, y_pred):
#     """Weighted Mean Squared Error to prioritize true negatives."""
#     weights = K.switch(y_true < 0.70, 2.0, 1.0)  # Weight true negatives higher
#     return K.mean(weights * K.square(y_true - y_pred))

# # Load the existing model
# model = load_model("/Users/cezar/Desktop/Team Project/AI/shotput/stage1/shotput_stage1.keras")

# Save the model with the updated TensorFlow version
model.save("/Users/cezar/Desktop/Team Project/AI/shotput/stage1/new_shotput_stage1.keras")


In [24]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input

def fix_and_save_model(input_model_path, output_model_path):
    """Fix model Input layer by removing `batch_shape` and re-save the model."""
    print(f"Loading model from {input_model_path}...")
    model = load_model(input_model_path)

    # Inspect and fix the Input layer configuration
    input_layer = model.layers[0]
    config = input_layer.get_config()

    if "batch_shape" in config:
        print("Fixing Input layer configuration...")
        config["shape"] = config.pop("batch_shape")[1:]  # Remove batch dimension
        new_input = Input(**config)

        # Rebuild the model
        x = new_input
        for layer in model.layers[1:]:
            x = layer(x)
        model = Model(inputs=new_input, outputs=x)

    # Save the updated model
    print("Saving fixed model to:", output_model_path)
    model.save(output_model_path)
    print("Model saved successfully.")



# Update paths as necessary
input_model_path = "/Users/cezar/Desktop/Team Project/AI/shotput/stage1/shotput_stage1.keras"
output_model_path = "/Users/cezar/Desktop/Team Project/AI/shotput/stage1/new_shotput_stage1.keras"

fix_model(input_model_path, output_model_path)


Loading model from /Users/cezar/Desktop/Team Project/AI/shotput/stage1/shotput_stage1.keras...
Saving updated model to /Users/cezar/Desktop/Team Project/AI/shotput/stage1/new_shotput_stage1.keras...
Model saved successfully.


In [28]:
from tensorflow.keras.models import load_model

model = load_model("/Users/cezar/Desktop/Team Project/AI/shotput/stage1/new_shotput_stage1.keras")
input_layer = model.layers[0]
print("Input Layer Config:", input_layer.get_config())


Input Layer Config: {'name': 'lstm_3', 'trainable': True, 'dtype': 'float32', 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'zero_output_for_mask': False, 'units': 64, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'recurrent_initializer': {'module': 'keras.initializers', 'class_name': 'OrthogonalInitializer', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.0, 'recurrent_dropout': 0.0, 'seed': None}

In [30]:
fixed_model = load_model("/Users/cezar/Desktop/Team Project/AI/shotput/stage1/new_shotput_stage1.keras")
input_layer = fixed_model.layers[0]
print("Fixed Input Layer Config:", input_layer.get_config())


Fixed Input Layer Config: {'name': 'lstm_3', 'trainable': True, 'dtype': 'float32', 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'zero_output_for_mask': False, 'units': 64, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'recurrent_initializer': {'module': 'keras.initializers', 'class_name': 'OrthogonalInitializer', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.0, 'recurrent_dropout': 0.0, 'seed':

In [33]:
from tensorflow.keras.models import load_model

model = load_model("/Users/cezar/Desktop/Team Project/AI/shotput/stage1/new_shotput_stage1.keras")
print("Model Config:", model.get_config())


Model Config: {'name': 'sequential_3', 'trainable': True, 'dtype': 'float32', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': (None, 169, 1), 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_3'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'LSTM', 'config': {'name': 'lstm_3', 'trainable': True, 'dtype': 'float32', 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'zero_output_for_mask': False, 'units': 64, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'recurrent_initializer': {'module': 'keras.initializers', 'class_name': 'OrthogonalInitializer', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'con

In [35]:
model = load_model("/Users/cezar/Desktop/Team Project/AI/shotput/stage1/new_shotput_stage1.keras")

# Save architecture and weights separately
model_config = model.to_json()
model_weights = model.get_weights()


In [37]:
from tensorflow.keras.models import model_from_json

# Rebuild the model without loading serialized batch_shape
new_model = model_from_json(model_config)  # Architecture only
new_model.set_weights(model_weights)  # Set weights

# Save the rebuilt model
new_model.save("/Users/cezar/Desktop/Team Project/AI/shotput/stage1/new_shotput_stage1.keras")


In [38]:
import logging
import traceback

# Import the main function from your existing script
from stage1_script import main  # Replace 'your_script_file' with the filename (without .py)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

def test_script():
    # Define the video path for testing
    VIDEO_PATH = "/Users/cezar/Desktop/Team Project/team-project/backend/evaluation-scripts/test_videos/stage1/videos/1_user5.mp4"

    logging.info("Starting test...")

    try:
        # Call the main function to test the process
        main(VIDEO_PATH)
    except FileNotFoundError as fnfe:
        logging.error(f"File not found: {fnfe}")
    except Exception as e:
        logging.error("An error occurred during testing:")
        logging.error(traceback.format_exc())

    logging.info("Test complete.")

if __name__ == "__main__":
    test_script()


2025-01-16 19:00:57,815 [INFO] Starting test...
I0000 00:00:1737050457.812716 14127678 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1737050457.928104 14316047 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737050457.943327 14316052 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Extracting keypoints...
Loading model...
Predicting score...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step


2025-01-16 19:00:58,453 [INFO] Test complete.


Results saved to stage1_results.json
